# Lead Scoring Prediction

An education company named X Education sells online courses to industry professionals. On any given day, many professionals who are interested in the courses land on their website and browse for courses.

The company markets its courses on several websites and search engines like Google. Once these people land on the website, they might browse the courses or fill up a form for the course or watch some videos. When these people fill up a form providing their email address or phone number, they are classified to be a lead. Moreover, the company also gets leads through past referrals. Once these leads are acquired, employees from the sales team start making calls, writing emails, etc. Through this process, some of the leads get converted while most do not. The typical lead conversion rate at X education is around 30%.

Now, although X Education gets a lot of leads, its lead conversion rate is very poor. For example, if, say, they acquire 100 leads in a day, only about 30 of them are converted. To make this process more efficient, the company wishes to identify the most potential leads, also known as ‘Hot Leads’. If they successfully identify this set of leads, the lead conversion rate should go up as the sales team will now be focusing more on communicating with the potential leads rather than making calls to everyone.

There are a lot of leads generated in the initial stage (top) but only a few of them come out as paying customers from the bottom. In the middle stage, you need to nurture the potential leads well (i.e. educating the leads about the product, constantly communicating, etc. ) in order to get a higher lead conversion.

X Education wants to select the most promising leads, i.e. the leads that are most likely to convert into paying customers. The company requires you to build a model wherein you need to assign a lead score to each of the leads such that the customers with higher lead score h have a higher conversion chance and the customers with lower lead score have a lower conversion chance. The CEO, in particular, has given a ballpark of the target lead conversion rate to be around 80%.

**Note**: The overview is taken directly from Kaggle: https://www.kaggle.com/datasets/amritachatterjee09/lead-scoring-dataset/data

In [ ]:
!pip install xplainable
!pip install xplainable-client

## Install Packages

In [1]:
!pip install xplainable
!pip install altair==5.4.1 #Upgrade this to work in Google Colab
!pip install xplainable-client
!pip install kaggle

## Package Imports

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import requests
import json

import xplainable as xp
from xplainable.core.models import XClassifier
from xplainable.core.optimisation.bayesian import XParamOptimiser
from xplainable.preprocessing.pipeline import XPipeline
from xplainable.preprocessing import transformers as xtf

from xplainable_client.client.client import XplainableClient
from xplainable_client.client.base import XplainableAPIError

## Instantiate Xplainable Cloud

Initialise the xplainable cloud using an API key from:
https://platform.xplainable.io/

This allows you to save and collaborate on models, create deployments, create shareable reports with a free trial of 14 days.

In [ ]:
# Initialize Xplainable Cloud client
client = XplainableClient(
    api_key="", #Create api key in xplainable cloud - https://platform.xplainable.io/
    hostname="https://platform.xplainable.io"
)

Connected to Xplainable Cloud
   User: jtuppa
   Hostname: http://localhost:8000
   API Key Expires: 2025-09-04T10:30:18.154043
   Python Version: 3.10.17
   Xplainable Version: 1.3.0


## Read Lead Scoring Dataset

**Note**: You can download the dataset to run this notebook from https://www.kaggle.com/datasets/amritachatterjee09/lead-scoring-dataset.

In [3]:
df = pd.read_csv('https://xplainable-public-storage.syd1.digitaloceanspaces.com/example_data/lead_scoring.csv')

### Sample of the Lead Scoring Dataset

In [4]:
df.head()

,Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,...,Get updates on DM Content,Lead Profile,City,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,No,No,0,0.0,0,0.0,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,No,No,0,5.0,674,2.5,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.0,...,No,Potential Lead,Mumbai,02.Medium,01.High,14.0,20.0,No,Yes,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.0,...,No,Select,Mumbai,02.Medium,01.High,13.0,17.0,No,No,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.0,...,No,Select,Mumbai,02.Medium,01.High,15.0,18.0,No,No,Modified


## 1. Data Preprocessing

In [5]:
#Instantiate a Pipeline
pipeline = XPipeline()

# Add stages for specific features
pipeline.add_stages([
    {"transformer": xtf.DropCols(
        columns=['Prospect ID', #-> Highly Cardinal, 
                 "Lead Number", #-> Reduce Multicollinearity between Tenure and Monthly Costs
                ]
    )},
])

### Preprocessed data

In [6]:
df_transformed = pipeline.fit_transform(df)
df_transformed.head()

,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,Specialization,...,Get updates on DM Content,Lead Profile,City,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity
0,API,Olark Chat,No,No,0,0.0,0,0.0,Page Visited on Website,Select,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Modified
1,API,Organic Search,No,No,0,5.0,674,2.5,Email Opened,Select,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Email Opened
2,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.0,Email Opened,Business Administration,...,No,Potential Lead,Mumbai,02.Medium,01.High,14.0,20.0,No,Yes,Email Opened
3,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.0,Unreachable,Media and Advertising,...,No,Select,Mumbai,02.Medium,01.High,13.0,17.0,No,No,Modified
4,Landing Page Submission,Google,No,No,1,2.0,1428,1.0,Converted to Lead,Select,...,No,Select,Mumbai,02.Medium,01.High,15.0,18.0,No,No,Modified


### Create Preprocessor ID to persist to Xplainable Cloud

In [ ]:

preprocessor_id, version_id = client.preprocessing.create_preprocessor(
    preprocessor_name="Lead Scoring Preprocessing",
    preprocessor_description="Predicting the Likelihood of a Lead Converting",
    pipeline=pipeline,
    df=df
)

In [8]:
preprocessor_id, version_id

('1IkLdZXzAEZfPJpE', '2yDjTGvCgVk0kmhE')

### Loading the Preprocessor steps
Use the api to load pre-existing preprocessor steps from the xplainable cloud and transform data inplace.

In [9]:

pp_cloud = client.preprocessing.load_preprocessor(
    version_id,
    preprocessor_id,
)

In [10]:
pp_cloud.stages

[{'feature': '__dataset__',
  'name': 'DropCols',
  'transformer': <xplainable.preprocessing.transformers.dataset.DropCols at 0x16fbe7e80>}]

In [11]:
df_transformed_cloud = pp_cloud.transform(df)


In [12]:
df_transformed_cloud

,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,Specialization,...,Get updates on DM Content,Lead Profile,City,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity
0,API,Olark Chat,No,No,0,0.0,0,0.00,Page Visited on Website,Select,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Modified
1,API,Organic Search,No,No,0,5.0,674,2.50,Email Opened,Select,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Email Opened
2,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.00,Email Opened,Business Administration,...,No,Potential Lead,Mumbai,02.Medium,01.High,14.0,20.0,No,Yes,Email Opened
3,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.00,Unreachable,Media and Advertising,...,No,Select,Mumbai,02.Medium,01.High,13.0,17.0,No,No,Modified
4,Landing Page Submission,Google,No,No,1,2.0,1428,1.00,Converted to Lead,Select,...,No,Select,Mumbai,02.Medium,01.High,15.0,18.0,No,No,Modified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9132,Landing Page Submission,Direct Traffic,Yes,No,1,8.0,1845,2.67,Email Marked Spam,IT Projects Management,...,No,Potential Lead,Mumbai,02.Medium,01.High,15.0,17.0,No,No,Email Marked Spam
9133,Landing Page Submission,Direct Traffic,No,No,0,2.0,238,2.00,SMS Sent,Media and Advertising,...,No,Potential Lead,Mumbai,02.Medium,01.High,14.0,19.0,No,Yes,SMS Sent
9134,Landing Page Submission,Direct Traffic,Yes,No,0,2.0,199,2.00,SMS Sent,Business Administration,...,No,Potential Lead,Mumbai,02.Medium,01.High,13.0,20.0,No,Yes,SMS Sent
9135,Landing Page Submission,Google,No,No,1,3.0,499,3.00,SMS Sent,Human Resource Management,...,No,NaN,Other Metro Cities,02.Medium,02.Medium,15.0,16.0,No,No,SMS Sent


### Create Train/Test split for model training validation

In [13]:
X, y = df_transformed.drop(columns=['Converted']), df['Converted']

#Optional if you want to use the cloud preprocessor
# X, y = df_transformed_cloud.drop(columns=['Converted']), df_transformed_cloud['Converted']

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.33, random_state=42)

## 2. Model Optimisation
The `XParamOptimiser` is utilised to fine-tune the hyperparameters of our model. This process searches for the optimal parameters that will yield the best model performance, balancing accuracy and computational efficiency.

In [14]:
opt = XParamOptimiser()
params = opt.optimise(X_train, y_train)

100%|██████████| 30/30 [00:03<00:00,  8.59trial/s, best loss: -0.8594330347239172]


## 3. Model Training
With the optimised parameters obtained, the `XClassifier` is trained on the dataset. This classifier undergoes a fitting process with the training data, ensuring that it learns the underlying patterns and can make accurate predictions.

In [15]:
model = XClassifier(**params)
model.fit(X_train, y_train)

## 4. Model Interpretability and Explainability
Following training, the `model.explain()` method is called to generate insights into the model's decision-making process. This step is crucial for understanding the factors that influence the model's predictions and ensuring that the model's behaviour is transparent and explainable.

In [16]:
model.explain()

alt.HConcatChart(...)

The visualization displays feature importance and contribution analysis for the lead scoring prediction model.

The 'Feature Importances' chart ranks features by their ability to predict lead conversion probability. Key factors influencing lead scoring typically include engagement metrics, demographic information, and behavioral patterns from the lead's interactions.

The 'Contributions' chart shows how specific feature values impact individual predictions, helping understand what drives lead qualification decisions and enabling sales teams to prioritize high-quality prospects.

## 5. Model Persisting
In this step, we first create a unique identifier for our churn prediction model using `client.create_model_id`. This identifier, shown as `model_id`, represents the newly instantiated model which predicts the likelihood of customers leaving within the next month. Following this, we generate a specific version of the model with `client.create_model_version`, passing in our training data. The output `version_id` represents this particular iteration of our model, allowing us to track and manage different versions systematically.


In [17]:
# Create a model
model_id, version_id = client.models.create_model(
    model=model,
    model_name="Lead Scoring",
    model_description="Predicting the likelihood of a lead converting",
    x=X_train,
    y=y_train
)

  0%|          | 0/32 [00:00<?, ?it/s]

## 6. Model Deployment

The code block illustrates the deployment of our churn prediction model using the `client.deployments.deploy` function. The deployment process involves specifying the `hostname` of the server where the model will be hosted, as well as the unique `model_id` and `version_id` that we obtained in the previous steps. This step effectively activates the model's endpoint, allowing it to receive and process prediction requests. The output confirms the deployment with a `deployment_id`, indicating the model's current `status` as 'inactive', its `location`, and the `endpoint` URL where it can be accessed for xplainable deployments.


In [18]:
if model_id and version_id:
    try:
        deployment_response = client.deployments.deploy(
            model_version_id=version_id #<- Use version id produced above
        )
        deployment_id = deployment_response.deployment_id
    except XplainableAPIError as e:
        print(f"Error deploying model: {e}")
        deployment_id = None
else:
    deployment_id = None

### Testing the Deployment programatically
This section demonstrates the steps taken to programmatically test a deployed model. These steps are essential for validating that the model's deployment is functional and ready to process incoming prediction requests.


1. **Activating the Deployment:**
   The model deployment is activated using `client.activate_deployment`, which changes the deployment status to active, allowing it to accept prediction requests.

In [19]:
if deployment_id:
    try:
        client.deployments.activate_deployment(deployment_id=deployment_id)
    except XplainableAPIError as e:
        print(f"Error activating deployment: {e}")
else:
    print("Deployment ID not available")

2. **Creating a Deployment Key:**
   A deployment key is generated with `client.deployments.generate_deploy_key`. This key is required to authenticate and make secure requests to the deployed model.

In [20]:
if deployment_id:
    try:
        deploy_key = client.deployments.generate_deploy_key(
            deployment_id=deployment_id, 
            description='API key for Lead Scoring Prediction',
            days_until_expiry=30
        )
        print(f"Deploy key created: {str(deploy_key)}")
    except XplainableAPIError as e:
        print(f"Error generating deploy key: {e}")
        deploy_key = None
else:
    deploy_key = None

Deploy key created: 3272a7e2-9c28-4c8d-a3b8-cc70d2168619


3. **Generating Example Payload:**
   An example payload for a deployment request is generated by `client.generate_example_deployment_payload`. This payload mimics the input data structure the model expects when making predictions.

In [21]:
#Set the option to highlight multiple ways of creating data
option = 2

In [22]:
df_transformed.columns

Index(['Lead Origin', 'Lead Source', 'Do Not Email', 'Do Not Call',
       'Converted', 'TotalVisits', 'Total Time Spent on Website',
       'Page Views Per Visit', 'Last Activity', 'Specialization',
       'How did you hear about X Education', 'What is your current occupation',
       'What matters most to you in choosing a course', 'Search', 'Magazine',
       'Newspaper Article', 'X Education Forums', 'Newspaper',
       'Digital Advertisement', 'Through Recommendations',
       'Receive More Updates About Our Courses', 'Lead Quality',
       'Update me on Supply Chain Content', 'Get updates on DM Content',
       'Lead Profile', 'City', 'Asymmetrique Activity Index',
       'Asymmetrique Profile Index', 'Asymmetrique Activity Score',
       'Asymmetrique Profile Score',
       'I agree to pay the amount through cheque',
       'A free copy of Mastering The Interview', 'Last Notable Activity'],
      dtype='object')

In [23]:
if option == 1 and version_id:
    try:
        body = client.deployments.generate_example_deployment_payload(
            model_version_id=version_id
        )
    except XplainableAPIError as e:
        print(f"Error generating example payload: {e}")
        body = []
else:
    body = json.loads(df_transformed.drop(columns=["Converted"]).sample(1).to_json(orient="records"))
    body[0]["Gender"] = None #<- Won't require this line the next release of xplainable

In [24]:
body

[{'Lead Origin': 'API',
  'Lead Source': 'Olark Chat',
  'Do Not Email': 'No',
  'Do Not Call': 'No',
  'TotalVisits': 0.0,
  'Total Time Spent on Website': 0,
  'Page Views Per Visit': 0.0,
  'Last Activity': 'SMS Sent',
  'Specialization': 'Select',
  'How did you hear about X Education': 'Select',
  'What is your current occupation': 'Unemployed',
  'What matters most to you in choosing a course': 'Better Career Prospects',
  'Search': 'No',
  'Magazine': 'No',
  'Newspaper Article': 'No',
  'X Education Forums': 'No',
  'Newspaper': 'No',
  'Digital Advertisement': 'No',
  'Through Recommendations': 'No',
  'Receive More Updates About Our Courses': 'No',
  'Lead Quality': None,
  'Update me on Supply Chain Content': 'No',
  'Get updates on DM Content': 'No',
  'Lead Profile': 'Select',
  'City': 'Select',
  'Asymmetrique Activity Index': None,
  'Asymmetrique Profile Index': None,
  'Asymmetrique Activity Score': None,
  'Asymmetrique Profile Score': None,
  'I agree to pay the amo

4. **Making a Prediction Request:**
   A POST request is made to the model's prediction endpoint with the example payload. The model processes the input data and returns a prediction response, which includes the predicted class (e.g., 'No' for no churn) and the prediction probabilities for each class.

In [ ]:
if deploy_key and body:
    response = requests.post(
        url="https://inference.xplainable.io/v1/predict",
        headers={'api_key': str(deploy_key)},
        json=body
    )

    value = response.json()
    print("Prediction result:", value)
else:
    print("Deploy key or body not available for prediction")

Prediction result: [{'index': 0, 'id': None, 'partition': '__dataset__', 'score': 0.5138461113290878, 'proba': 0.5402210582390248, 'pred': 1, 'support': 735, 'breakdown': [{'feature': 'base_value', 'value': None, 'score': 0.3806567554321189}, {'feature': 'Lead Origin', 'value': 'API', 'score': -0.010196328908133758}, {'feature': 'Lead Source', 'value': 'Olark Chat', 'score': -0.013443347877402288}, {'feature': 'Do Not Email', 'value': 'No', 'score': 0.004493683317418156}, {'feature': 'Do Not Call', 'value': 'No', 'score': -4.199526127277487e-05}, {'feature': 'TotalVisits', 'value': '0', 'score': 0.0058464572979032665}, {'feature': 'Total Time Spent on Website', 'value': '0', 'score': 0.005248030353684144}, {'feature': 'Page Views Per Visit', 'value': '0', 'score': 0.0058464572979032665}, {'feature': 'Last Activity', 'value': 'SMS Sent', 'score': 0.038624526799564095}, {'feature': 'Specialization', 'value': 'Select', 'score': 0.005284430725558849}, {'feature': 'How did you hear about X 

: 